In [63]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

In [131]:
df = pd.read_json('world_bank_projects.json')

In [132]:
#Find the 10 countries with most projects

df.countryname.value_counts().head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

In [139]:
# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

# load json file as a string, and then normalize the mjtheme_namecode column so that it can be viewed in dataframe form
json_string = json.load((open('world_bank_projects.json')))
normal = json_normalize(json_string, 'mjtheme_namecode')

In [146]:
# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')
normal.name.value_counts()

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: name, dtype: int64

In [147]:
# 3. In 2. above you will notice that some entries have only the code and the name is missing. 
#   Create a dataframe with the missing names filled in.


#create copy of normal df, so as to use as a lookup table
lookup = normal.copy()

In [148]:
# in the name column, replace any blank values with nan
lookup['name'] = lookup['name'].replace('', np.nan)

lookup = lookup.dropna()

In [149]:
#remove duplicates rows, so that the lookup table only has one value per code column
lookup = lookup.drop_duplicates()

In [150]:
print(lookup)

    code                                          name
0      8                             Human development
2      1                           Economic management
3      6         Social protection and risk management
4      5                         Trade and integration
5      2                      Public sector governance
6     11  Environment and natural resources management
8      7                   Social dev/gender/inclusion
11     4      Financial and private sector development
18    10                             Rural development
53     9                             Urban development
252    3                                   Rule of law


In [151]:
print(normal.head())

  code                                   name
0    8                      Human development
1   11                                       
2    1                    Economic management
3    6  Social protection and risk management
4    5                  Trade and integration


In [152]:
#merge lookup table with "normal" data frame, the merged dataframe is called xy
xy = pd.merge(normal, lookup, on ='code')


In [153]:
#this shows that the merged xy dataframe now has 2 name columns.  The name_x column is the original one with blank values,
# while name_y is fully populated with the names
print(xy.head())

  code             name_x             name_y
0    8  Human development  Human development
1    8                     Human development
2    8  Human development  Human development
3    8  Human development  Human development
4    8  Human development  Human development


In [154]:
# drop the name_x column since we don't need it anymore; the 1 is the axis to drop, i.e. column, while 0 means rows.
xy = xy.drop('name_x',1)

In [155]:
print(xy.head())

  code             name_y
0    8  Human development
1    8  Human development
2    8  Human development
3    8  Human development
4    8  Human development


In [156]:
# rename the name_y column to name
xy.columns = ['code','name']

In [157]:
#Find the top 10 major project themes (using column 'mjtheme_namecode')
top10 = xy.name.value_counts().sort_values(ascending = 0).head(10)

In [158]:
print(top10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64
